In [ ]:
pip install transformers datasets evaluate seqeval

In [ ]:
! pip install -U accelerate
! pip install -U transformers

In [ ]:
!pip install transformers[torch]
!pip install accelerate

In [ ]:
pip install accelerate -U

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
import evaluate
import numpy as np
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import torch

In [ ]:
wnut = load_dataset("wnut_17")

In [ ]:
wnut

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [ ]:
wnut["train"]

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 3394
})

In [ ]:
wnut["train"][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [ ]:
wnut["train"].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product'], id=None), length=-1, id=None)}

In [ ]:
wnut["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product'], id=None), length=-1, id=None)

In [ ]:
# B harfi bir varligin baslangicini gosterir, I harfi ayni varligin icinde bir belirtec oldugunu gosterir, O ise herhangi bir varliga karsilik gelmedigini gosterir.
# Buradaki cikti olan array kodu calistiran kisiler tarafindan eldeki veri ve tespit edilmesi istenen kisimlar baz alinarak
# arastirmacilar ya da bu isle ugrasan kisiler tarafindan tasarlanmalidir.
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [ ]:
# burada AutoTokenizer.from_pretrained("bert-base-cased") yerine uncased olan modeli kullaniyoruz eger modelimiz buyuk kucuk harf
# onemseyen bir veriye sahipsek cased olani kullanabiliriz ama buyuk kucuk harf bizim icin onemli olmadigi icin uncased modelini kullaniyoruz.
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
example = wnut["train"][0]
example["tokens"]

['@paulwalk',
 'It',
 "'s",
 'the',
 'view',
 'from',
 'where',
 'I',
 "'m",
 'living',
 'for',
 'two',
 'weeks',
 '.',
 'Empire',
 'State',
 'Building',
 '=',
 'ESB',
 '.',
 'Pretty',
 'bad',
 'storm',
 'here',
 'last',
 'evening',
 '.']

In [ ]:
# tokenizer fonksiyonu metin verisini modelin isleyebilecegi bir hale getirir metni daha kucuk parcalara (token) boler
# bu tokenlerin ise her biri belirli bir kelime veya sembolu ifade eden sayi dizisidir.
# is_split_into_words parametresi bu metnin kelime duzeyinde bir ayrima gittigini ifade eder.
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokenized_input

{'input_ids': [101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# fonksiyon isminden de anlasilacagi gibi ustteki input_ids kisimlarini kelimelere cevirir, skip_special_tokens parametresi default olarak false ayarlanmistir
# eger skip_special_tokens parametresini True yaparsak CLS ve SEP degerleri gosterilmez
# CLS(classification) baslangic, (SEP) separator ise sonlandirmak icin kullanilir
# '@paulwalk' kismi '@', 'paul' ve '##walk' olarak 3 kisma ayrildi buradaki ## isareti kelimenin bir oncekinin devami oldugunu belirtir ve
# bu islem BERT modelinin kelimenin yapisin daha iyi anlamasini saglar.
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 '@',
 'paul',
 '##walk',
 'it',
 "'",
 's',
 'the',
 'view',
 'from',
 'where',
 'i',
 "'",
 'm',
 'living',
 'for',
 'two',
 'weeks',
 '.',
 'empire',
 'state',
 'building',
 '=',
 'es',
 '##b',
 '.',
 'pretty',
 'bad',
 'storm',
 'here',
 'last',
 'evening',
 '.',
 '[SEP]']

In [ ]:
tokenized_inputs = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
tokenized_inputs

{'input_ids': [101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
word_ids = tokenized_input.word_ids()
tokenized_input, word_ids

({'input_ids': [101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 [None,
  0,
  0,
  0,
  1,
  2,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  None])

In [ ]:
# tokenizer icinde truncation=True parametresi BERT modelinin anlayacabilecegi maksimum uzunlukta olacak sekilde kirpma islemi yapar.
# word_ids, batch_index birden fazla veriyi ayni anda islerken hangi tokenlerin hangi kelimelere ait oldugunu belirler.
# ['[CLS]', '@', 'paul', '##walk', 'it', "'", 's', 'the', 'view', 'from', 'where', 'i', "'", 'm', 'living', 'for', 'two', 'weeks', '.', 'empire', 'state', 'building', '=', 'es', '##b', '.', 'pretty', 'bad', 'storm', 'here', 'last', 'evening', '.', '[SEP]']
# bu fonksiyonun amaci bir ornek alip ornegi tokenlerine ayirmak ve bu tokenleri de etiketleri ile hizalamaktir, etiket hizalamasi ise her bir tokenin metinde hangi kelimeye
# karsilik geldigini anlamamiz icin kullanilir.

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        # '@paulwalk' -> '@', 'paul', '##walk'
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                # add the corresponding token
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
example[f"ner_tags"]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 8,
 8,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
q = tokenize_and_align_labels(wnut["train"][0:1])
q

{'input_ids': [[101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, -100, -100, 0, 0, -100, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, -100, 0, 0, 0, 0, 0, 0, 0, 0, -100]]}

In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q["input_ids"][0]), q["labels"][0]):
  print(f"{token:_<40} {label} ")

[CLS]___________________________________ -100 
@_______________________________________ 0 
paul____________________________________ -100 
##walk__________________________________ -100 
it______________________________________ 0 
'_______________________________________ 0 
s_______________________________________ -100 
the_____________________________________ 0 
view____________________________________ 0 
from____________________________________ 0 
where___________________________________ 0 
i_______________________________________ 0 
'_______________________________________ 0 
m_______________________________________ -100 
living__________________________________ 0 
for_____________________________________ 0 
two_____________________________________ 0 
weeks___________________________________ 0 
._______________________________________ 0 
empire__________________________________ 7 
state___________________________________ 8 
building________________________________ 8 
=________________

In [ ]:
# wnut.map islemi düzensiz ve gürültülü metinlerde dogru bir sekilde haritalamak icin kullanilir
# ornek "2moro" ifadesi tomorrow yani DATE olarak siniflandirmak gerek bu gorevi ustlenir.
# bu fonksiyon parametre olarak verilen fonksiyonu her bir ornek uzerine uygulamaya yarar.
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True) # batched true tek tek islemek yerine toplu islemeye yardimci olur.

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [ ]:
# bu fonksiyon default olarak padding=True parametresine sahiptir bu parametre padding islemine gerek olup olmadigini kontrol eder isledigimiz cumleler farklı
# uzunlukta olabilecegi icin bu parametre onemlidir ve butun cumleleri ayni uzunluktaymis gibi duzenler ve bu programin daha verimli calismasina katki saglar(en uzun ornek uzunluguna tamamlar)
# buradaki tamamlama islemi [PAD] eklenerek yapilir
# bu fonksiyon veri hazirlama islemlerini kolaylastirmak icin kullanilir
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
# 1. Girdi verilerini tokenlara ayırır ve modelin anlayabileceği formata getirir.
# 2. Tokenlara ilişkin etiketleri hizalar. Yani her bir tokenın hangi etiketi taşıdığını belirler.
# 3. Uzunlukları farklı olan örnekleri aynı uzunluğa getirir (padding işlemi).

In [ ]:
# seqeval bir Python kütüphanesidir ve sıralı veri değerlendirme (sequence evaluation) işlemlerini gerçekleştirmek için kullanılır
# NER gibi sirali veri etiketleme islerinde tahminler ile gercek degerleri karsilastirmada kullanilir.
# Bu kütüphane, modelin sıralı veri üzerindeki performansını değerlendirmek için kullanışlıdır ve
# genellikle girdi olarak iki liste alır: tahmin edilen etiketler ve gerçek etiketler
seqeval = evaluate.load("seqeval")

In [ ]:
labels = [label_list[i] for i in example[f"ner_tags"]]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-location',
 'I-location',
 'I-location',
 'O',
 'B-location',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
seqeval.compute(predictions=[labels], references=[labels])

{'location': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
# bu fonksiyona verilen p parametresi bir tuple olmalidir ve predictions, labels ikilisini icermelidir
# predictions modelin tahmin ettigi etiket olasilik dagilimlarini icerir
# np.argmax fonksiyonu tahmin edilenlerden maximumu alır axis=2 olmasinin sebebi ise Model, her bir token için birden fazla etiket olasılığını tahmin eder ve axis=2 de boyutu temsil eder
"""Metin:      New York şehri güzel bir yer.
   Etiketler:  B-LOCATION   I-LOCATION O O O O O
Tahminler: [
 [0.1, 0.05, 0.2, 0.2, 0.2, 0.05, 0.1],
 [0.2, 0.1, 0.05, 0.2, 0.2, 0.15, 0.1],
 [0.1, 0.15, 0.2, 0.2, 0.1, 0.05, 0.2],  # çok boyutlu oldugu icin argmax=2
 [0.2, 0.2, 0.15, 0.2, 0.1, 0.05, 0.1],
 [0.2, 0.2, 0.1, 0.2, 0.05, 0.05, 0.15],
 [0.15, 0.2, 0.1, 0.2, 0.1, 0.05, 0.2],
 [0.1, 0.05, 0.2, 0.2, 0.15, 0.1, 0.2],
]
"""
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    # seqeval.compute fonksiyonu precision, recall, f1-score ve accuracy degerlerini dondurur hepsini return ediyoruz.
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
# etiketleri sayısal indekslere ve sayısal indeksleri etiketlere dönüştürmek için kullanılır.
# Model tahmini ya da gercek degerler baz alindiginda bunlari etiketlere donusturmek icin id2label kullanilir.
# Verilen gercek etiketleri sayisal indekslere donusturmek icin ise label2id kullanilir.
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

In [ ]:
# Otomatik olarak belirli bir gorevi gerceklestirmek icin onceden egitilmis bir dil modeli ve biz burada bert_base_uncased kullaniyoruz.
# Token tabanli sirali etiketleme gorevi icin ozel bir siniflandirma katmanini iceren bir modeldir.
# len(labels) = 13 oldugu icin yani ner_tags toplam 13 tane oldugu icin num_labels olarak 13 degerini veriyoruz.
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-uncased", num_labels=13, id2label=id2label, label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`
training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model", # buradaki output_dir modelin kaydedilecegi klasoru belirtir
    learning_rate=2e-5, # Adam optimizer icin learning rate ayari yapilir
    per_device_train_batch_size=16, # Egitim islemi sırasında her cihazda (GPU veya TPU) kullanılacak egitim orneklerinin sayısını belirler.
    per_device_eval_batch_size=16, # Degerlendirme islemi sırasında her cihazda kullanılacak degerlendirme orneklerinin sayısını belirler.
    num_train_epochs=2, # Egitim dongusundeki toplam epoch sayisini belirtir artirirsak daha iyi egitmis olabiliriz fakat cok zaman alir.
    weight_decay=0.01, # bu parametre overfittingden korumak icin kullanilir
    evaluation_strategy="epoch", # degerlendirme stratejisi olarak gecer, her epoch sonunda bir degerlendirme sonucu yayinlar.
    save_strategy="epoch", # her epoch sonunda model kaydedilir.
    load_best_model_at_end=True, # model sonunda en iyi modeli en iyi sonuclu olan modeli yukler.
    push_to_hub=False,
)

trainer = Trainer(
    model=model, # egitilecek modeli belirler ustte belirttigimiz gibi BERT modeli kullaniyoruz.
    args=training_args, # bu parametre ustte yazdigimiz arguments kismi model hakkinda learning_rate, epochs vs. belirtir.
    train_dataset=tokenized_wnut["train"], # egitim veri kumesi
    eval_dataset=tokenized_wnut["test"], # degerlendirme veri kumesi
    tokenizer=tokenizer,
    data_collator=data_collator, # veri ornekleme islemini gerceklestirecek islevi belirtir.
    compute_metrics=compute_metrics, # bu yukarida yazdigimiz fonksiyon precision, recall, f1-score ve accuracy degerlerini donduruyoruz.
)

trainer.train() # egitim dongusunu baslatir.

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.299724,0.520677,0.256719,0.343886,0.938181
2,No log,0.283561,0.579470,0.324374,0.415924,0.942285


TrainOutput(global_step=426, training_loss=0.20547861448476012, metrics={'train_runtime': 118.0474, 'train_samples_per_second': 57.502, 'train_steps_per_second': 3.609, 'total_flos': 184389735625176.0, 'train_loss': 0.20547861448476012, 'epoch': 2.0})

In [ ]:
# ornek text
text = "The Golden State Warriors are an American professional basketball team based in San Francisco."

In [ ]:
# Bu pipeline Hugging Face Transformers kutuphanesi tarafindan olusturulmus bir varlik(orneğin kisiler, kuruluslar, konumlar) tanima modelidir.
from transformers import pipeline

classifier = pipeline("ner", model="stevhliu/my_awesome_wnut_model")
classifier(text)

[{'entity': 'B-location',
  'score': 0.42658585,
  'index': 2,
  'word': 'golden',
  'start': 4,
  'end': 10},
 {'entity': 'I-location',
  'score': 0.35856324,
  'index': 3,
  'word': 'state',
  'start': 11,
  'end': 16},
 {'entity': 'B-group',
  'score': 0.3064002,
  'index': 4,
  'word': 'warriors',
  'start': 17,
  'end': 25},
 {'entity': 'B-location',
  'score': 0.6552352,
  'index': 13,
  'word': 'san',
  'start': 80,
  'end': 83},
 {'entity': 'B-location',
  'score': 0.46686625,
  'index': 14,
  'word': 'francisco',
  'start': 84,
  'end': 93}]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_wnut_model")
inputs = tokenizer(text, return_tensors="pt")

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("stevhliu/my_awesome_wnut_model")
with torch.no_grad(): # tells PyTorch to not calculate the gradients, egitim kismini gectigimiz ve tahmin kismina geldigimiz icin bunu no_grad() yapiyoruz.
    logits = model(**inputs).logits #  Logitler, farklı etiket sınıflarının olasılık skorlarını temsil eder.
    # .logits ise logitlere erisim saglar ve olasilik skorlarini iceren bir tensor dondurur.

In [ ]:
predictions = torch.argmax(logits, dim=2)
# Bu ifade, predictions tensöründeki ilk (muhtemelen tek bir örnek için) girdinin tahmin edilen
# sınıf indekslerini gerçek sınıf etiketlerine dönüştürür. Her indeks id2label sözlüğünde bulunan gerçek sınıf etiketiyle eşleştirilir.
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
predicted_token_class

['O',
 'O',
 'B-location',
 'I-location',
 'B-group',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-location',
 'B-location',
 'O',
 'O']